In [2]:
import numpy as np
import pandas as pd
import json 

In [3]:
# f = open(r'./data/shipsnet.json')
# json.load(f)
labels_json = pd.read_json('./data/shipsnet.json')

In [4]:
# Example image name: 0__20160820_233143_0c53__-122.3294838764191_37.8004805108676
labels_json.head()

,data,labels,locations,scene_ids
0,"[82, 89, 91, 87, 89, 87, 86, 86, 86, 86, 84, 8...",1,"[-118.2254694333423, 33.73803725920789]",20180708_180909_0f47
1,"[76, 75, 67, 62, 68, 72, 73, 73, 68, 69, 69, 6...",1,"[-122.33222866289329, 37.7491755586813]",20170705_180816_103e
2,"[125, 127, 129, 130, 126, 125, 129, 133, 132, ...",1,"[-118.14283073363218, 33.736016066914175]",20180712_211331_0f06
3,"[102, 99, 113, 106, 96, 102, 105, 105, 103, 10...",1,"[-122.34784341495181, 37.76648707436548]",20170609_180756_103a
4,"[78, 76, 74, 78, 79, 79, 79, 82, 86, 85, 83, 8...",1,"[-122.34852408322172, 37.75878462398653]",20170515_180653_1007


In [5]:
rgbs = labels_json['data']
r = rgbs[:6400]
g = rgbs[6400:6400*2]
b = rgbs[6400*2:]
len(r), len(g), len(b)

rgbs.dtype

dtype('O')